In [1]:
from quantum_mcmc_routines import *

In [13]:
# define the model
np.random.seed(1)# should always be in the same cell!  
n_spins = 4

## construct problem Hamiltonian ##
shape_of_J=(n_spins,n_spins)

# defining J matrix (mutual 1-1 interaction)
J =  np.round(np.random.randn(n_spins,n_spins), decimals=2)#np.random.uniform(low= -1, high= 1, size= (n_spins, n_spins) )
J = 0.5 * (J + J.transpose() )
# print("J before:"); print(J)
J= J - np.diag(np.diag(J))

# J=np.array([[0,1,0.5,-1],[1,0,0.3,0.5],[0.5,0.3,0,1],[-1,0.5,1,0]])
print("J after:", J)

# defining h
h = np.round(np.random.randn(n_spins), decimals=2)#np.random.uniform(low= -1, high = 1, size= (n_spins))
# h=np.array([0.5]*n_spins)
print("h is:", h)

# instantiate the model
model = IsingEnergyFunction(J, h)
# print(model.get_energy('0100'))
alpha = np.sqrt(n_spins) / np.sqrt( sum([J[i][j]**2 for i in range(n_spins) for j in range(i)]) + sum([h[j]**2 for j in range(n_spins)])  )
print("alpha: ", alpha)

J after: [[ 0.     0.13  -0.105 -0.695]
 [ 0.13   0.     0.745 -0.57 ]
 [-0.105  0.745  0.    -0.465]
 [-0.695 -0.57  -0.465  0.   ]]
h is: [-0.17 -0.88  0.04  0.58]
alpha:  1.206396381047913


In [15]:
a,b,c=quantum_enhanced_mcmc(
    N_hops=10,
    num_spins=4,
    initial_state='1011',
    num_elems=16,
    model=model,
    alpha=alpha,
    return_last_n_states=10,
    return_both=True,
    temp=0.4,
)

runnning quantum MCMC steps . ..:  10%|█         | 1/10 [00:00<00:00,  9.83it/s]

starting with:  1011


runnning quantum MCMC steps . ..: 100%|██████████| 10/10 [00:01<00:00,  7.49it/s]


In [18]:
print(b); print(c)

['1101', '1000', '1001', '1001', '0001', '1010', '0000', '0001', '0000', '1110']
['1101', '1101', '1101', '1101', '1101', '1101', '1101', '1101', '0000', '0000']
